# ITER TBR Calculation
Goal : Build a working ITER TBR Simulation

Name : Husni Naufal Zuhdi (413821)

In [1]:
# Impor semua paket yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show)
from jupyter_cadquery import set_sidecar, set_defaults, reset_defaults

import openmc
import paramak
import paramak_neutronics as nparamak
import neutronics_material_maker as nmm

Overwriting auto display for cadquery Workplane and Shape


## Step 1 : Define Materials

The materials use in this study are listed below
1. Plasma = Deuterium and Tritium
2. Blanket Fluid = LiF
3. Neutron Multiplier = Natural Pb
4. Moderator = Graphite
5. Reflector = Graphite
6. First Wall, Divertor, dan Vaccum Vessel= SS 316

In [45]:
# Plasma Material
plasma_mat = nmm.Material(
    name='U235',
    isotopes={'U235':1},
    density=1,
    percent_type='ao',
    density_unit='g/cm3'
)
plasma_mat.openmc_material

Material
	ID             =	36
	Name           =	U235
	Temperature    =	None
	Density        =	1.0 [g/cm3]
	S(a,b) Tables  
	Nuclides       
	U235           =	1            [ao]

In [3]:
# Blanket Fluid Material
enrichment_Li = 10
blanket_fluid_mat = nmm.Material.from_library(name='Lithium Fluoride',
                                        enrichment=enrichment_Li,
                                        enrichment_target='Li6',
                                        enrichment_type='ao'
                                       )
blanket_fluid_mat.openmc_material

Material
	ID             =	4
	Name           =	Lithium Fluoride
	Temperature    =	None
	Density        =	2.635 [g/cm3]
	S(a,b) Tables  
	Nuclides       
	Li6            =	0.05         [ao]
	Li7            =	0.45         [ao]
	F19            =	0.5          [ao]

In [4]:
# Neutron Multiplier Material
neutron_multi_mat = nmm.Material.from_library(name='Lead')
neutron_multi_mat.openmc_material

Material
	ID             =	6
	Name           =	Lead
	Temperature    =	None
	Density        =	11.35 [g/cm3]
	S(a,b) Tables  
	Nuclides       
	Pb208          =	0.524        [ao]
	Pb204          =	0.014        [ao]
	Pb207          =	0.221        [ao]
	Pb206          =	0.241        [ao]

In [5]:
# Moderator Material
blanket_mod_mat = nmm.Material.from_library(name="Carbon, Graphite (reactor grade)")
blanket_mod_mat.openmc_material

Material
	ID             =	8
	Name           =	Carbon, Graphite (reactor grade)
	Temperature    =	None
	Density        =	1.7 [g/cm3]
	S(a,b) Tables  
	Nuclides       
	B11            =	8.017999999999999e-07 [ao]
	B10            =	1.982e-07    [ao]
	C12            =	0.9889210110779999 [ao]
	C13            =	0.011077988922 [ao]

In [6]:
# Moderator and Reflector Materials
blanket_ref_mat = nmm.Material.from_library(name="Carbon, Graphite (reactor grade)")
blanket_ref_mat.openmc_material

Material
	ID             =	10
	Name           =	Carbon, Graphite (reactor grade)
	Temperature    =	None
	Density        =	1.7 [g/cm3]
	S(a,b) Tables  
	Nuclides       
	B11            =	8.017999999999999e-07 [ao]
	B10            =	1.982e-07    [ao]
	C12            =	0.9889210110779999 [ao]
	C13            =	0.011077988922 [ao]

In [7]:
# First Wall Material
blanket_first_wall_mat = nmm.Material.from_library('Steel, Stainless 316')
blanket_first_wall_mat.openmc_material

Material
	ID             =	12
	Name           =	Steel, Stainless 316
	Temperature    =	None
	Density        =	8.0 [g/cm3]
	S(a,b) Tables  
	Nuclides       
	C12            =	0.0018789517999999999 [ao]
	C13            =	2.10482e-05  [ao]
	Si28           =	0.0092672382464 [ao]
	Si30           =	0.00031019783679999996 [ao]
	Si29           =	0.00047056391679999997 [ao]
	P31            =	0.000413     [ao]
	S33            =	1.946594e-06 [ao]
	S36            =	3.7908e-08   [ao]
	S34            =	1.0909573999999999e-05 [ao]
	S32            =	0.000247105924 [ao]
	Cr53           =	0.01729048986 [ao]
	Cr52           =	0.15248424954 [ao]
	Cr50           =	0.007907291700000001 [ao]
	Cr54           =	0.004303968900000001 [ao]
	Mn55           =	0.010274     [ao]
	Fe57           =	0.014129725090000001 [ao]
	Fe56           =	0.6118257649400001 [ao]
	Fe58           =	0.00188040702 [ao]
	Fe54           =	0.03897510295000001 [ao]
	Ni61           =	0.001297240397 [ao]
	Ni62           =	0.004136170035 [ao]


In [8]:
# Divertor Material
divertor_mat = nmm.Material.from_library('Steel, Stainless 316')
divertor_mat.openmc_material

Material
	ID             =	14
	Name           =	Steel, Stainless 316
	Temperature    =	None
	Density        =	8.0 [g/cm3]
	S(a,b) Tables  
	Nuclides       
	C12            =	0.0018789517999999999 [ao]
	C13            =	2.10482e-05  [ao]
	Si28           =	0.0092672382464 [ao]
	Si30           =	0.00031019783679999996 [ao]
	Si29           =	0.00047056391679999997 [ao]
	P31            =	0.000413     [ao]
	S33            =	1.946594e-06 [ao]
	S36            =	3.7908e-08   [ao]
	S34            =	1.0909573999999999e-05 [ao]
	S32            =	0.000247105924 [ao]
	Cr53           =	0.01729048986 [ao]
	Cr52           =	0.15248424954 [ao]
	Cr50           =	0.007907291700000001 [ao]
	Cr54           =	0.004303968900000001 [ao]
	Mn55           =	0.010274     [ao]
	Fe57           =	0.014129725090000001 [ao]
	Fe56           =	0.6118257649400001 [ao]
	Fe58           =	0.00188040702 [ao]
	Fe54           =	0.03897510295000001 [ao]
	Ni61           =	0.001297240397 [ao]
	Ni62           =	0.004136170035 [ao]


In [9]:
# Vaccum Vessel Material
vac_vessel_mat = nmm.Material.from_library('Steel, Stainless 316')
vac_vessel_mat.openmc_material

Material
	ID             =	16
	Name           =	Steel, Stainless 316
	Temperature    =	None
	Density        =	8.0 [g/cm3]
	S(a,b) Tables  
	Nuclides       
	C12            =	0.0018789517999999999 [ao]
	C13            =	2.10482e-05  [ao]
	Si28           =	0.0092672382464 [ao]
	Si30           =	0.00031019783679999996 [ao]
	Si29           =	0.00047056391679999997 [ao]
	P31            =	0.000413     [ao]
	S33            =	1.946594e-06 [ao]
	S36            =	3.7908e-08   [ao]
	S34            =	1.0909573999999999e-05 [ao]
	S32            =	0.000247105924 [ao]
	Cr53           =	0.01729048986 [ao]
	Cr52           =	0.15248424954 [ao]
	Cr50           =	0.007907291700000001 [ao]
	Cr54           =	0.004303968900000001 [ao]
	Mn55           =	0.010274     [ao]
	Fe57           =	0.014129725090000001 [ao]
	Fe56           =	0.6118257649400001 [ao]
	Fe58           =	0.00188040702 [ao]
	Fe54           =	0.03897510295000001 [ao]
	Ni61           =	0.001297240397 [ao]
	Ni62           =	0.004136170035 [ao]


## Langkah 2 : Definisikan Geometri

In [46]:
# Color list : https://www.tug.org/pracjourn/2007-4/walden/color.pdf
# Make a ITER Tokamak Geometry Class
class ITERTokamak_mod(paramak.Reactor):
    """Create ITER geometry without TF Coils and PF Coils. This class modified
    from ITERTokamak function from paramak package"""
    
    def __init__(
        self,
        rotation_angle: float = 360.,
    ):
        super().__init__([])
        
        self.rotation_angle = rotation_angle
    
    def create_vessel_components(self) -> list:
        """Creates a 3d solids for each vessel component.

        Returns:
            A list of CadQuery solids: A list of 3D solid volumes
        """
        
        # Blanket reflector
        blanket_ref = paramak.BlanketFP(
            plasma=self.plasma,
            thickness=15,
            start_angle=-70,
            stop_angle=230,
            rotation_angle=self.rotation_angle,
            vertical_displacement=self.plasma.vertical_displacement,
            offset_from_plasma=[[-70, 0, 90, 180, 230], [85, 55, 94, 51, 85]],
            name='blanket_ref | Graphite',
            color=(0.9,0.4,0),
            stp_filename="blanket_ref.stp",
            stl_filename="blanket_ref.stl",
            material_tag='blanket_ref_mat',
        )
        
        # Blanket Fluid
        blanket_fluid = paramak.BlanketFP(
            plasma=self.plasma,
            thickness=30,
            start_angle=-70,
            stop_angle=230,
            rotation_angle=self.rotation_angle,
            vertical_displacement=self.plasma.vertical_displacement,
            offset_from_plasma=[[-70, 0, 90, 180, 230], [55, 25, 64, 21, 55]],
            name='blanket_fluid | LIF',
            color=(0.9,0.9,0),
            stp_filename="blanket_fluid.stp",
            stl_filename="blanket_fluid.stl",
            material_tag='blanket_fluid_mat',
        )

        # Blanket first wall
        blanket_first_wall = paramak.BlanketFP(
            plasma=self.plasma,
            thickness=5,
            start_angle=-70,
            stop_angle=230,
            rotation_angle=self.rotation_angle,
            vertical_displacement=self.plasma.vertical_displacement,
            offset_from_plasma=[[-70, 0, 90, 180, 230], [50, 20, 59, 16, 50]],
            name='blanket_first_wall | SS 316',
            color=(0.6,0.7,1.0),
            stp_filename="blanket_first_wall.stp",
            stl_filename="blanket_first_wall.stl",
            material_tag='blanket_first_wall_mat',
        )

        # SN Divertor
        divertor = paramak.ITERtypeDivertor(
            anchors=((4.34e2, -3.3e2), (5.56e2, -3.74e2)),
            coverages=(105, 125),
            lengths=(45, 75),
            radii=(68, 65),
            tilts=(-30, 2),
            dome_height=45,
            dome_pos=0.45,
            rotation_angle=self.rotation_angle,
            name='divertor | SS 316',
            color=(0.3,0,0.6),
            stp_filename="divertor.stp",
            stl_filename="divertor.stl",
            material_tag='divertor_mat',
        )

        # Vacuum vessel
        divertor.points  # trigger the building of the points for divertor
        # the inner part of the vacuum vessel is computed from the outer
        # points of the blanket and the divertor
        vac_vessel_inner = paramak.RotateMixedShape(
            points=blanket_ref.outer_points + divertor.casing_points,
            rotation_angle=self.rotation_angle,
        )

        vac_vessel = paramak.RotateSplineShape(
            points=[
                (327.77, 36.5026668124882),
                (327.77, 73.37741270075162),
                (327.77, 108.31180820215741),
                (327.77, 143.2462037035632),
                (327.77, 178.18059920496898),
                (327.77, 213.11499470637477),
                (327.77, 248.04939020778068),
                (327.77, 282.98378570918646),
                (327.77, 317.9181812105922),
                (328.6121587814181, 368.23899806938385),
                (336.18303032328333, 422.4306297110355),
                (350.4835654579176, 457.5437492206628),
                (371.95910957013655, 492.47041663587777),
                (404.3208742000702, 522.0151685493631),
                (439.6516080621078, 544.4559826211985),
                (474.98234192414554, 556.3610266211815),
                (510.2245275810152, 564.0927634387052),
                (545.6438096482208, 565.1200145185009),
                (565.832800426528, 563.1864687746993),

                (580.9745435102584, 559.4390362932862),
                (616.3052773722961, 548.4109567158157),
                (651.6360112343338, 533.224020531035),
                (686.9667450963714, 515.3041214328789),
                (722.297478958409, 492.23516177329117),
                (757.6282128204466, 466.8689289401416),
                (792.9589466824843, 437.10619055069265),
                (825.7660566972336, 403.7167485984509),
                (853.525919017406, 369.42176700251196),
                (877.9209495411939, 333.90960594986575),
                (898.9511482685972, 300.5186330502012),
                (916.616515199616, 265.2383422522439),
                (932.5994662324425, 230.72194441870647),
                (946.0587934179808, 193.1122328856627),
                (956.1532888071343, 156.87835598377137),
                (962.8829523999035, 118.10702768634405),
                (967.9302000944803, 80.39197257542594),
                (968.7714080435763, 38.24754419835381),

                (968.7714080435763, 25.77097437642317),
                (964.5653682980957, -1.670738783514139),
                (956.9944967562304, -29.93883090626548),
                (956.1532888071343, -34.59540221679083),
                (946.0587934179808, -71.15339839027786),
                (931.7582582833464, -104.25874435511184),
                (914.9340993014238, -139.91477225259314),
                (898.9511482685972, -174.48160361826422),
                (883.8094051848669, -213.64300914878197),
                (867.8264541520404, -248.21908241802464),
                (851.0022951701176, -284.2078188440911),
                (834.1781361881949, -319.9470238737184),
                (818.1951851553683, -359.0978394110024),
                (800.5298182243495, -391.2313539579658),
                (776.1347877005617, -427.87174371008393),
                (744.1688856349085, -460.45530873911446),
                (708.8381517728709, -490.0255912806248),
                (673.5074179108332, -512.7040543014494),
                (638.1766840487956, -528.371873327094),
                (602.8459501867579, -539.0490644239661),
                (567.5152163247203, -546.1219131278361),
                (532.1844824626827, -548.9566889080664),
                (496.85374860064496, -547.7514325554811),
                (461.52301473860734, -541.3971156414638),
                (426.1922808765697, -527.596464992453),
                (390.8615470145321, -501.2796363633471),
                (360.57806084707124, -468.0473902249954),
                (340.389070068764, -431.4355817359209),
                (329.87397070506233, -399.072068113844),
                (327.770950832322, -357.4796824533661),
                (327.770950832322, -311.73270913617455),
                (327.770950832322, -276.79831363476876),
                (327.770950832322, -241.86391813336297),
                (327.770950832322, -206.92952263195718),
                (327.770950832322, -171.99512713055117),
                (327.770950832322, -137.06073162914538),
                (327.770950832322, -102.12633612773948),
                (327.770950832322, -67.19194062633369),

            ],
            cut=[vac_vessel_inner],  # to make a hollow shape
            rotation_angle=self.rotation_angle,
            name='vac_vessel | SS 316',
            color=(0.9,0.9,0.9),
            stp_filename='vac_vessel.stp',
            stl_filename='vac_vessel.stl',
            material_tag='vac_vessel_mat',
        )

        return [divertor, blanket_fluid, blanket_ref, blanket_first_wall, vac_vessel, vac_vessel_inner]
    
    
    def create_plasma(self) -> list:
        """Creates a 3d solids for the plasma.

        Returns:
            A list of CadQuery solids: A list of 3D solid volumes
        """

        self.plasma = paramak.Plasma(
            major_radius=620,
            minor_radius=200,
            elongation=1.7,
            triangularity=0.33,
            vertical_displacement=5.7e1,
            configuration="single-null",
            rotation_angle=self.rotation_angle,
            name='plasma | U235',
            color=(0.7,0,0.7),
            stp_filename='plasma.stp',
            stl_filename='plasma.stl',
            material_tag='plasma_mat',
        )

        return [self.plasma]
    
    def create_solids(self):
        """Creates a 3d solids for each component.

        Returns:
            A list of CadQuery solids: A list of 3D solid volumes
        """

        plasma = self.create_plasma()
        vessel = self.create_vessel_components()

        shapes_and_components = plasma + vessel[:-1]
        self.shapes_and_components = shapes_and_components

        return shapes_and_components

In [44]:
# Show the reactor geometry
my_reactor = ITERTokamak_mod()
my_reactor.show()

In [47]:
# Export dagmc.h5m file
my_reactor.export_h5m()

Saved file as  divertor.stl
Saved file as  blanket_fluid.stl
Saved file as  blanket_ref.stl
Saved file as  blanket_first_wall.stl
Saved file as  vac_vessel.stl
Saved file as  graveyard.stl


'dagmc.h5m'

In [86]:
help(my_reactor.export_h5m)

Help on method export_h5m in module paramak.reactor:

export_h5m(filename: Union[str, NoneType] = 'dagmc.h5m', include_graveyard: Union[bool, NoneType] = True, include_plasma: Union[bool, NoneType] = False, method: Union[str, NoneType] = None, merge_tolerance: Union[float, NoneType] = None, faceting_tolerance: Union[float, NoneType] = None) -> str method of __main__.ITERTokamak_mod instance
    Produces a h5m neutronics geometry compatable with DAGMC
    simulations. Tags the volumes with their material_tag attributes. Sets
    the Reactor.h5m_filename to the filename of the h5m file produced.
    
    Arguments:
        filename: filename of h5m outputfile.
        include_graveyard: specifiy if the graveyard will be included or
            not. If True the the Reactor.make_graveyard will be called
            using Reactor.graveyard_size and Reactor.graveyard_offset
            attribute values.
        method: The method to use when making the imprinted and
            merged geomet

## Langkah 3 : Definisikan Sumber

In [60]:
# Define a ring source
source = openmc.Source() # initialises a new source object
radius = openmc.stats.Discrete([545, 570, 620, 670, 695], [1, 1, 1, 1, 1]) # the distribution of source radius values is just a single value
z_values = openmc.stats.Discrete([-25, -15, -5, 5, 15, 25], [0.9, 1, 1, 1, 1, 0.9]) # the distribution of source z values is just a single value
angle = openmc.stats.Uniform(a=0., b=2* 3.14159265359) # the distribution of source azimuthal angles values is a uniform distribution between 0 and 2 Pi
source.space = openmc.stats.CylindricalIndependent(r=radius, phi=angle, z=z_values, origin=(0.0, 0.0, 0.0)) # this makes the ring source using the three distributions and a radius
source.angle = openmc.stats.Isotropic() # sets the direction to isotropic
# source.energy = openmc.stats.Discrete([14.08e6], [1.0]) # sets the discrite energy distribution at 14.08 MeV
source.energy = openmc.stats.Discrete([0.005], [1.0])
source.particle = 'neutron'

## Langkah 4 : Definisikan Tally TBR dan Jalankan Perhitungan OpenMC

In [61]:
k_plasma_mat = plasma_mat
k_plasma_mat.name = 'plasma_mat'

k_divertor_mat = divertor_mat
k_divertor_mat.name = 'divertor_mat'

k_blanket_fluid_mat = blanket_fluid_mat
k_blanket_fluid_mat.name = 'blanket_fluid_mat'

k_blanket_ref_mat = blanket_ref_mat
k_blanket_ref_mat.name = 'blanket_ref_mat'

k_blanket_first_wall_mat = blanket_first_wall_mat
k_blanket_first_wall_mat.name = 'blanket_first_wall_mat'

k_vac_vessel_mat = vac_vessel_mat
k_vac_vessel_mat.name = 'vac_vessel_mat'

In [62]:
# Calculate k value
# Pack materials into one variable
mats = openmc.Materials([k_plasma_mat.openmc_material,
                         k_divertor_mat.openmc_material,
                         k_blanket_fluid_mat.openmc_material,
                         k_blanket_ref_mat.openmc_material,
                         k_blanket_first_wall_mat.openmc_material,
                         k_vac_vessel_mat.openmc_material])

In [80]:
# Get geometry from dagmc.h5m file
dag_universe = openmc.DAGMCUniverse(filename="dagmc.h5m", auto_geom_ids=True, auto_mat_ids=True)
geom = openmc.Geometry(root=dag_universe)

In [64]:
# Instantiate a Settings object
sett = openmc.Settings()
sett.particles = 1000 # for each generation
sett.generations_per_batch = 1000
sett.batches = 50
sett.inactive = 5
sett.dagmc = True
sett.verbosity = 10
sett.source = source

In [65]:
# Run OpenMC!
model = openmc.model.Model(geom, mats, sett)
model.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

RuntimeError: No fission sites banked on MPI rank 0

In [ ]:
# open the results file
sp = openmc.StatePoint('statepoint.'+str(batches)+'.h5')